## Notebook 6: Levenshtein Distance for Streets

In this Jupyter Notebook the derived street names from the OCR are compared to a ground truth list with street names in Nuremberg after 1870.

### Import libraries

1. **re**  - This is a library to process regular expressions. 

2. **Numpy** - Numpy is a library for the easy use of vectors, matrices or arrays in general. It simplifies various numerical operations. 

3. **Codecs** - This module provides access to the most common Python encoders and decoders for example to be used for text encoding.

4. **Pandas** - Pandas is a library to analyze and to manage data. It is used to create tables.

5. **Levenshtein** -  The levensthein_distance is imported and used as a similarity measure

In [2]:
!pip install Levenshtein
'''Import Statements'''
import pandas as pd
import re
import numpy as np
from numpy import nan
from Levenshtein import distance as levenshtein_distance
# from google.colab import drive 
# drive.mount('/content/gdrive')
import io

### Import a Street Name list

In [4]:
'''Creation of a streetnames list as a reference vocabulary for Street.
The normalized occupation is added into the column "Normalized Street Name"'''

#streetnames = pd.read_csv('/Users/ilona-dewikusardi/Desktop/Datensets/streetlist.xlsx - Tabelle1.csv',usecols=["REFERENZNAMEN", "NAMENSVARIANTEN AB 1870", "ÄLTERE BEZEICHNUNGEN"])
streetnames = pd.read_csv('./files/Reference_Vocabularies/streetlist.xlsx - Tabelle1.csv',usecols=["REFERENZNAMEN", "NAMENSVARIANTEN AB 1870", "ÄLTERE BEZEICHNUNGEN"])
streetnames["NAMENSVARIANTEN AB 1870"] = streetnames["NAMENSVARIANTEN AB 1870"] + "; " + streetnames["ÄLTERE BEZEICHNUNGEN"]
streetnames = streetnames.drop(columns=["ÄLTERE BEZEICHNUNGEN"])
streetnames = streetnames.apply(lambda x: x.str.split(';').explode()).drop_duplicates().reset_index().drop(columns=["index"])

streetnameGroundTruth = streetnames
#streetnames.head(50)

In [3]:
#person_df = pd.read_csv('/Users/ilona-dewikusardi/Desktop/Datensets/F_23_92.jpg/2_person_df.csv', lineterminator='\n', dtype={'Street': str,'House Number': int, 'House Floor': str, 'House Owner': str, 'Last Name': str, 'First Name Abbreviation': str, 'Owner Street': str, 'Owner Number': str, 'Occupation': str, 'Add Info': str})
#person_df = pd.read_csv('/content/gdrive/MyDrive/MA Python/Outputs/2_person_df.csv', lineterminator='\n', dtype={'Street': str,'House Number': int, 'House Floor': str, 'House Owner': str, 'Last Name': str, 'First Name Abbreviation': str, 'Owner Street': str, 'Owner Number': str, 'Occupation': str, 'Add Info': str})
person_df = pd.read_csv('./Outputs/2_person_df.csv', lineterminator='\n', dtype={'Full Name': str, 'Last Name': str, 'First Name Abbreviation': str, 'Occupation': str, 'Occupation 1': str, 'Occupation 2': str,'Add Info': str, 'House Owner': str, 'Full Address': str, 'Street': str, 'House Number': str, 'Part of House': str, 'House Floor': str, 'Building': str, 'BuildingPart': str, 'BuildingPartFloor': str, 'Full Owner Address': str, 'Owner Street': str, 'Owner Number': str})
person_df = person_df.fillna('')

#company_df = pd.read_csv('/Users/ilona-dewikusardi/Desktop/Datensets/F_23_92.jpg/2_company_df.csv', lineterminator='\n', dtype={'Street': str,'House Number': int, 'House Owner': str, 'Company Name': str, 'Owner Street': str, 'Owner Number': str})
#company_df = pd.read_csv('/content/gdrive/MyDrive/MA Python/Outputs/2_company_df.csv', lineterminator='\n', dtype={'Street': str,'House Number': int, 'House Owner': str, 'Company Name': str, 'Owner Street': str, 'Owner Number': str})
company_df = pd.read_csv('./Outputs/2_company_df.csv', lineterminator='\n', dtype={'Company Name': str, 'Industry': str, 'Full Address': str, 'Street': str, 'House Number': str, 'Part of House': str, 'House Floor': str, 'House Owner': str, 'Full Owner Address': str, 'Owner Street': str, 'Owner Number': str})
company_df = company_df.fillna('')

#Creating a subset of the df for working
#df = person_df.iloc[:200]
df = person_df

### Comparison and Correction of the Street Names

In [4]:
def streetAbbrCorrection(table, column1, word):
    if word is not None:
        #table[column1] = table[column1].apply(lambda x: re.sub(r'(\w*)str\.?\b',r'\g<1>straße', str(x)))
        table[column1] =  [re.sub(r'(\w*)str\.',r'\1straße', str(x)) for x in table[column1]]
        return table

for idxAdrBook, word in company_df['Owner Street'].iteritems():
    if word is not None:
        word = str(word)
        company_df = streetAbbrCorrection(company_df, 'Owner Street', word)

#company_df.head(20)

In [5]:
Beruf_ohne_Normb = []

Levenshtein_1 = []
Levenshtein_2 = []
Levenshtein_3 = []

'''Wenn das word (=Straßenname im DataFrame) in der Liste professions (in der Spalte Berufe) vorliegt, 
dann übernimm in Spalte "Normalized Occupation" im DataFrame den Inhalt der Spalte Normbezeichnungen aus der professions-Liste.
Wenn Normbezeichnungen leer, dann üvernimm Inhalt von OhdAB_01, etc.'''

def streetCorrection(table, column1, column2, column3, word, streetnames):
    if word is not None:
        word = word.strip()
        if word in streetnames['REFERENZNAMEN'].values:
            table.loc[table[column1] == (word), column2] = (word)
            #table.loc[table['Owner Street'] == (word), 'Normalized Owner Street Name'] = (word)
            return table
        if word in streetnames['NAMENSVARIANTEN AB 1870'].values:
            idx = np.where(streetnames['NAMENSVARIANTEN AB 1870'] == word)[0][0]
            table.loc[table[column1] == (word), column2] = (streetnames['REFERENZNAMEN'][idx])
            #table.loc[table['Owner Street'] == (word), 'Normalized Owner Street Name'] = (streetnames['REFERENZNAMEN'][idx])
            #table[column] = table[column].replace(word, streetnames['REFERENZNAMEN'][idx])
            return table
        else:
            table.loc[table[column1] == (word), column3] = (word)
        
    return table
   
#df['Normalized Streetname'] = ''
#df['Streetname not found'] = ''

for idxAdrBook, word in df['Street'].iteritems():
    word = str(word)
    df = streetCorrection(df, 'Street', 'Normalized Street', 'Streetname not found', word, streetnameGroundTruth)

for idxAdrBook, word in df['Owner Street'].iteritems():
    if word is not None:
        word = str(word)
        df = streetCorrection(df, 'Owner Street', 'Normalized Owner Street', 'Streetname not found 2', word, streetnameGroundTruth)

for idxAdrBook, word in company_df['Street'].iteritems():
    word = str(word)
    company_df = streetCorrection(company_df, 'Street', 'Normalized Street', 'Streetname not found', word, streetnameGroundTruth)

for idxAdrBook, word in company_df['Owner Street'].iteritems():
    if word is not None:
        word = str(word)
        company_df = streetCorrection(company_df, 'Owner Street', 'Normalized Owner Street', 'Streetname not found 2', word, streetnameGroundTruth)

#for idxAdrBook, word in company_df['Owner Street'].iteritems():
#    word = str(word)
#    company_df = streetCorrection(company_df, 'Owner Street', word, streetnameGroundTruth)    
#company_df.head(50)

In [6]:
Unknown_Streetname = []

def LevenshteinStreet(table, column, column_1, column_2, column_3, column_4, column_5, word, streetnames):
    for idx2, streetname2 in streetnames['REFERENZNAMEN'].iteritems():
        streetname2 = str(streetname2).strip()
        #print('Word: ' + (word) + ' ' + 'Streetname: ' + (streetname2) + ' ' + str(levenshtein_distance(word, streetname2)))
        if word !='':
            if levenshtein_distance(word, streetname2) == 1 and not word in streetnames['REFERENZNAMEN']:
                #print('Word: ' + (word) + ' ' + 'Street: ' + (streetname) + ' ' + str(levenshtein_distance(word, streetname)))
                table.loc[table[column] == (word), column_1] = (streetnames['REFERENZNAMEN'][idx2])
                table.loc[table[column] == (word), column_3] = (streetnames['REFERENZNAMEN'][idx2])
                table.loc[table[column] == (word), column_2] = None
                Levenshtein_1.append(streetnames['REFERENZNAMEN'][idx2])
                return table
            elif levenshtein_distance(word, streetname2) == 2 and not word in streetnames['REFERENZNAMEN']:
                table.loc[table[column] == (word), column_1] = (streetnames['REFERENZNAMEN'][idx2])
                table.loc[table[column] == (word), column_4] = (streetnames['REFERENZNAMEN'][idx2])
                table.loc[table[column] == (word), column_2] = None
                Levenshtein_2.append(streetnames['REFERENZNAMEN'][idx2])
                return table
#            elif levenshtein_distance(word, streetname2) == 3 and not word in streetnames['REFERENZNAMEN']:
#                table.loc[table['Street'] == (word), 'Normalized Street'] = (streetnames['REFERENZNAMEN'][idx2])
#                table.loc[table['Street'] == (word), 'LS_Street_3'] = (streetnames['REFERENZNAMEN'][idx2])
#                table.loc[table['Street'] == (word), 'Streetname not found'] = None
#                Levenshtein_3.append(streetnames['REFERENZNAMEN'][idx2])
#                return table
            else:
                Unknown_Streetname.append(word)
    return table
    for idx, streetname in streetnames['NAMENSVARIANTEN AB 1870'].iteritems():
        streetname = str(streetname).strip()
        if word !='':
            if levenshtein_distance(word, streetname) == 1 and not word in streetnames['NAMENSVARIANTEN AB 1870']:
                #print('Word: ' + (word) + ' ' + 'Street: ' + (streetname) + ' ' + str(levenshtein_distance(word, streetname)))
                table.loc[table[column] == (word), column_1] = (streetnames['REFERENZNAMEN'][idx])
                table.loc[table[column] == (word), column_3] = (streetnames['REFERENZNAMEN'][idx])
                table.loc[table[column] == (word), column_2] = None
                Levenshtein_1.append(streetnames['REFERENZNAMEN'][idx])
                return table
            elif levenshtein_distance(word, streetname) == 2 and not word in streetnames['NAMENSVARIANTEN AB 1870']:
                table.loc[table[column] == (word), column_1] = (streetnames['REFERENZNAMEN'][idx])
                table.loc[table[column] == (word), column_4] = (streetnames['REFERENZNAMEN'][idx])
                table.loc[table[column] == (word), column_2] = None
                Levenshtein_2.append(streetnames['REFERENZNAMEN'][idx])
                return table
#            elif levenshtein_distance(word, streetname) == 3 and not word in streetnames['NAMENSVARIANTEN AB 1870']:
#                table.loc[table['Street'] == (word), 'Normalized Street'] = (streetnames['REFERENZNAMEN'][idx])
#                table.loc[table['Street'] == (word), 'LS_Street_3'] = (streetnames['REFERENZNAMEN'][idx])
#                table.loc[table['Street'] == (word), 'Streetname not found'] = None
#                Levenshtein_3.append(streetnames['REFERENZNAMEN'][idx])
#                return table
            else:
                Unknown_Streetname.append(word)
    return table

df['LS_Street_1'] = ''
df['LS_Street_2'] = ''
df['LS_Street_3'] = ''
df['LS_Street_4'] = ''
df['LS_Street_5'] = ''
df['LS_Street_6'] = ''

for idxAdrBook, word in df['Streetname not found'].iteritems():
    if word is not None:
        word = str(word)
        LevenshteinStreet(df, 'Street', 'Normalized Street', 'Streetname not found', 'LS_Street_1', 'LS_Street_2', 'LS_Street_3', word, streetnameGroundTruth)

for idxAdrBook, word in df['Streetname not found 2'].iteritems():
    if word is not None:
        word = str(word)
        LevenshteinStreet(df, 'Owner Street', 'Normalized Owner Street', 'Streetname not found 2', 'LS_Street_4', 'LS_Street_5', 'LS_Street_6', word, streetnameGroundTruth)

for idxAdrBook, word in company_df['Streetname not found'].iteritems():
    if word is not None:
        word = str(word)
        LevenshteinStreet(company_df, 'Street', 'Normalized Street', 'Streetname not found', 'LS_Street_1', 'LS_Street_2', 'LS_Street_3', word, streetnameGroundTruth)

for idxAdrBook, word in company_df['Streetname not found 2'].iteritems():
    if word is not None:
        word = str(word)
        LevenshteinStreet(company_df, 'Owner Street', 'Normalized Owner Street', 'Streetname not found 2', 'LS_Street_4', 'LS_Street_5', 'LS_Street_6', word, streetnameGroundTruth)

#company_df.head(50)
df.head(50)

,Full Name,Last Name,First Name Abbreviation,Occupation,Occupation 1,Occupation 2,Add Info,House Owner,Full Address,Street,House Number,Part of House,House Floor,Building,BuildingPart,BuildingPartFloor,Full Owner Address,Owner Street,Owner Number,Streetname not found,Normalized Street,Streetname not found 2,Normalized Owner Street,LS_Street_1,LS_Street_2,LS_Street_3,LS_Street_4,LS_Street_5,LS_Street_6
0,G. Pirner,Pirner,G.,Wirt,,,(zur Siegesgöttin),True,"Altcrstraße 1, Vorderhaus",Altcrstraße,1,Vorderhaus,,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus",,,,Altcrstraße,NaN,,NaN,,,,,,
1,G. Bogner,Bogner,G.,Maurerpaller,,,,False,"Altcrstraße 1, Vorderhaus, 0",Altcrstraße,1,Vorderhaus,0,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus, 0",,,,Altcrstraße,NaN,,NaN,,,,,,
2,G. Haßmann,Haßmann,G.,Lackierer,,,,False,"Altcrstraße 1, Vorderhaus, 1",Altcrstraße,1,Vorderhaus,1,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus, 1",,,,Altcrstraße,NaN,,NaN,,,,,,
3,K. Frühbeißer,Frühbeißer,K.,Kleiderm,,,,False,"Altcrstraße 1, Vorderhaus",Altcrstraße,1,Vorderhaus,,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus",,,,Altcrstraße,NaN,,NaN,,,,,,
4,J. tzrundler,tzrundler,J.,Feingoldschl,,,,False,"Altcrstraße 1, Vorderhaus",Altcrstraße,1,Vorderhaus,,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus",,,,Altcrstraße,NaN,,NaN,,,,,,
5,I. Pirner,Pirner,I.,Eisendreher,,,,False,"Altcrstraße 1, Vorderhaus, 2",Altcrstraße,1,Vorderhaus,2,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus, 2",,,,Altcrstraße,NaN,,NaN,,,,,,
6,G. Wagner,Wagner,G.,Taglöhner,,,,False,"Altcrstraße 1, Vorderhaus, 4",Altcrstraße,1,Vorderhaus,4,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus, 4",,,,Altcrstraße,NaN,,NaN,,,,,,
7,P. Kortes,Kortes,P.,Schlosser,,,,False,"Altcrstraße 1, Vorderhaus, 4",Altcrstraße,1,Vorderhaus,4,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus, 4",,,,Altcrstraße,NaN,,NaN,,,,,,
8,G. Röder,Röder,G.,Kutscher,,,,False,"Altcrstraße 1, Vorderhaus, 4",Altcrstraße,1,Vorderhaus,4,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus, 4",,,,Altcrstraße,NaN,,NaN,,,,,,
9,T Mandlinger,Mandlinger,T,Zimmges,,,,False,"Altcrstraße 1, Vorderhaus",Altcrstraße,1,Vorderhaus,,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus",,,,Altcrstraße,NaN,,NaN,,,,,,


In [7]:
'''If no normalized Streetname was found, insert the original streetname into the normalized column'''
'''
def using_fillna(table, column1, column2):
    table[column1] = table[column1].fillna(table[column2])
    return table

for idxAdrBook, word in df['Normalized Street'].iteritems():
    using_fillna(df, 'Normalized Street', 'Street')

for idxAdrBook, word in company_df['Normalized Street'].iteritems():
    using_fillna(company_df, 'Normalized Street', 'Street')

for idxAdrBook, word in company_df['Normalized Owner Street'].iteritems():
    using_fillna(company_df, 'Normalized Owner Street', 'Owner Street')
'''
df.head(50)

,Full Name,Last Name,First Name Abbreviation,Occupation,Occupation 1,Occupation 2,Add Info,House Owner,Full Address,Street,House Number,Part of House,House Floor,Building,BuildingPart,BuildingPartFloor,Full Owner Address,Owner Street,Owner Number,Streetname not found,Normalized Street,Streetname not found 2,Normalized Owner Street,LS_Street_1,LS_Street_2,LS_Street_3,LS_Street_4,LS_Street_5,LS_Street_6
0,G. Pirner,Pirner,G.,Wirt,,,(zur Siegesgöttin),True,"Altcrstraße 1, Vorderhaus",Altcrstraße,1,Vorderhaus,,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus",,,,Altcrstraße,NaN,,NaN,,,,,,
1,G. Bogner,Bogner,G.,Maurerpaller,,,,False,"Altcrstraße 1, Vorderhaus, 0",Altcrstraße,1,Vorderhaus,0,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus, 0",,,,Altcrstraße,NaN,,NaN,,,,,,
2,G. Haßmann,Haßmann,G.,Lackierer,,,,False,"Altcrstraße 1, Vorderhaus, 1",Altcrstraße,1,Vorderhaus,1,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus, 1",,,,Altcrstraße,NaN,,NaN,,,,,,
3,K. Frühbeißer,Frühbeißer,K.,Kleiderm,,,,False,"Altcrstraße 1, Vorderhaus",Altcrstraße,1,Vorderhaus,,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus",,,,Altcrstraße,NaN,,NaN,,,,,,
4,J. tzrundler,tzrundler,J.,Feingoldschl,,,,False,"Altcrstraße 1, Vorderhaus",Altcrstraße,1,Vorderhaus,,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus",,,,Altcrstraße,NaN,,NaN,,,,,,
5,I. Pirner,Pirner,I.,Eisendreher,,,,False,"Altcrstraße 1, Vorderhaus, 2",Altcrstraße,1,Vorderhaus,2,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus, 2",,,,Altcrstraße,NaN,,NaN,,,,,,
6,G. Wagner,Wagner,G.,Taglöhner,,,,False,"Altcrstraße 1, Vorderhaus, 4",Altcrstraße,1,Vorderhaus,4,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus, 4",,,,Altcrstraße,NaN,,NaN,,,,,,
7,P. Kortes,Kortes,P.,Schlosser,,,,False,"Altcrstraße 1, Vorderhaus, 4",Altcrstraße,1,Vorderhaus,4,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus, 4",,,,Altcrstraße,NaN,,NaN,,,,,,
8,G. Röder,Röder,G.,Kutscher,,,,False,"Altcrstraße 1, Vorderhaus, 4",Altcrstraße,1,Vorderhaus,4,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus, 4",,,,Altcrstraße,NaN,,NaN,,,,,,
9,T Mandlinger,Mandlinger,T,Zimmges,,,,False,"Altcrstraße 1, Vorderhaus",Altcrstraße,1,Vorderhaus,,Altcrstraße 1,"Altcrstraße 1, Vorderhaus","Altcrstraße 1, Vorderhaus",,,,Altcrstraße,NaN,,NaN,,,,,,


In [8]:
'''Storing the DataFrames with the Normalized Street and the Normalized Owner Street and all additional columns'''
#df.to_csv('/Users/ilona-dewikusardi/Desktop/Datensets/2_person_df_str.csv', index = False)
df.to_csv('./Outputs/2_person_df_str.csv', index = False)
df.to_csv('./Outputs/2_person_df_str_clean.csv',columns=['Full Name', 'Last Name', 'First Name Abbreviation', 'Occupation', 'Occupation 1', 'Occupation 2','Add Info', 'House Owner', 'Full Address', 'Street', 'Normalized Street', 'House Number', 'Part of House', 'House Floor', 'Building', 'BuildingPart', 'BuildingPartFloor', 'Full Owner Address', 'Owner Street', 'Normalized Owner Street', 'Owner Number'], index = False)

#company_df.to_csv('/Users/ilona-dewikusardi/Desktop/Datensets/2_company_df_str.csv', index = False)
company_df.to_csv('./Outputs/2_company_df_str.csv', index = False)

#cleaned company df
#company_df.to_csv('/content/gdrive/MyDrive/MA Python/Outputs/2_company_df_clean.csv',columns=['Street', 'Normalized Street', 'House Number', 'Company Name', 'Add Info', 'Owner Street', 'Normalized Owner Street', 'Owner Number'], index = False)
#company_df.to_csv('/Users/ilona-dewikusardi/Desktop/2_company_df_clean.csv',columns=['Company Name', 'Industry', 'Full Address', 'Street', 'Normalized Street', 'House Number', 'Part of House', 'House Floor', 'House Owner', 'Full Owner Address', 'Owner Street', 'Normalized Owner Street', 'Owner Number'], index = False)
company_df.to_csv('./Outputs/2_company_df_clean.csv',columns=['Company Name', 'Industry', 'Full Address', 'Street', 'Normalized Street', 'House Number', 'Part of House', 'House Floor', 'House Owner', 'Full Owner Address', 'Owner Street', 'Normalized Owner Street', 'Owner Number'], index = False)


list(set(Unknown_Streetname))
with open('./Outputs/Unknown Steetnames.txt','w') as f:
  f.write('\n'.join(set(Unknown_Streetname)))